In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import csv

In [2]:

# Initialization
url = 'https://www.france-renouvelables.fr/annuaire-des-membres/'

# Open Chrome driver
driver = webdriver.Chrome()
#driver = webdriver.Firefox() 

# Navigate to the URL
driver.get(url)


In [3]:
# Click on the acceptance button only once
try:
    cookie_banner = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'tarteaucitronPersonalize2'))
    )
    cookie_banner.click()
except Exception as e:
    print(f"Unable to click the 'Accept All Cookies' button for {url}: {str(e)}")


In [ ]:
# # Wait until the elements are loaded
# time.sleep(5)

# # Function to handle scrolling
# def custom_scroll():
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(3)
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.66);")

# # Main loop
# last_height = driver.execute_script("return document.body.scrollHeight")
# while True:
#     custom_scroll()
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if round(new_height * 0.9) >= last_height:
#         break
#     last_height = new_height


In [6]:
# Get the source and parse it
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

In [7]:
soup

<html class="js" lang="fr-FR" style="height: auto;"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="https://www.france-renouvelables.fr/xmlrpc.php" rel="pingback"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/>
<!-- This site is optimized with the Yoast SEO plugin v22.3 - https://yoast.com/wordpress/plugins/seo/ -->
<title>Annuaire des membres - France Renouvelables</title>
<link href="https://www.france-renouvelables.fr/annuaire-des-membres/" rel="canonical"/>
<meta content="fr_FR" property="og:locale"/>
<meta content="article" property="og:type"/>
<meta content="Annuaire des membres - France Renouvelables" property="og:title"/>
<meta content="https://www.france-renouvelables.fr/annuaire-des-membres/" property="og:url"/>
<meta content="France Renouvelables" property="og:site_name"/>
<meta content="https://www.facebook.com/fr.renouvelables/" property="article:publisher"/>
<m

In [13]:
MEMBER_DATA = []

for MEMBER in soup.find_all("div", {"class": "contenu"}):
    MEMBER_NAME = MEMBER.find("div", {"class": "nom_membre"}).text
    ADDRESS = MEMBER.find("div", {"class": "adresse"})
    ADDRESS_LINK = ADDRESS.a["href"] if ADDRESS else ""
    ADDRESS_TEXT = ADDRESS.text.strip() if ADDRESS else ""

    PHONE = MEMBER.find("div", {"class": "numero_telephone"})
    PHONE_VALUE = PHONE.a["title"] if PHONE else ""

    EMAIL = MEMBER.find("div", {"class": "email"})
    EMAIL_ADDRESS = EMAIL.a["href"].split("mailto:")[-1] if EMAIL else ""

    WEBSITE = MEMBER.find("div", {"class": "siteweb"})
    WEBSITE_LINK = WEBSITE.a["href"] if WEBSITE else ""

    MEMBER_DATA.append(
        {
            "Member Name": MEMBER_NAME,
            "Address": {"Link": ADDRESS_LINK, "Text": ADDRESS_TEXT},
            "Phone": PHONE_VALUE,
            "Email": EMAIL_ADDRESS,
            "Website": WEBSITE_LINK,
        }
    )

print(MEMBER_DATA)

[{'Member Name': '0X2', 'Address': {'Link': 'https://www.google.fr/maps/place/102 boulevard Sebastopol, 75003 Paris/', 'Text': ''}, 'Phone': '', 'Email': 'france@ox2.com', 'Website': 'https://www.ox2.com'}, {'Member Name': '2W RH', 'Address': {'Link': 'https://www.google.fr/maps/place/Savoie Technolac, 19 Allée du Lac Saint-André - 73372 LE BOURGET DU LAC CEDEX/', 'Text': ''}, 'Phone': '09 70 21 01 06', 'Email': 'contact@2w-rh.fr', 'Website': 'http://www.2w-rh.fr/'}, {'Member Name': '3D Energies SAEML', 'Address': {'Link': 'https://www.google.fr/maps/place/336 avenue de Paris 79000 Niort/', 'Text': ''}, 'Phone': '+33549252500', 'Email': 'info@3denergies.com', 'Website': 'https://www.3denergies.fr'}, {'Member Name': '3E France', 'Address': {'Link': 'https://www.google.fr/maps/place/1 rue du Poids de l’Huile 31000 Toulouse/', 'Text': ''}, 'Phone': '05 81 18 07 70', 'Email': 'info@3e.eu', 'Website': 'http://www.3e.eu/'}, {'Member Name': '8.2 France', 'Address': {'Link': 'https://www.googl

In [14]:
import csv
import pandas as pd


# Define headers
headers = ["Member Name", "Address", "Phone", "Email", "Website"]

# Convert scraped_data to DataFrame
df = pd.DataFrame(MEMBER_DATA, columns=headers)

# Export DataFrame to CSV
df.to_csv("footeolien.csv", index=False)